In [2]:
!pip install langchain
!pip install chromadb
!pip install pypdf
!pip install sentence-transformers
!pip install langchain-community langchain-core
!pip install openai
!pip install openpyxl



In [3]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from openai import OpenAI
import openpyxl



In [5]:
loaders = [PyPDFLoader("/content/CWEexplanations.pdf")]

docs = []

for file in loaders:
  docs.extend(file.load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
docs = text_splitter.split_documents(docs)
embedding_function = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2", model_kwargs = {'device': 'cpu'})

vectorstore = Chroma.from_documents(docs, embedding_function, persist_directory = "./chroma_db_nccn")
print(vectorstore._collection.count())

<ipython-input-5-4b01d539f8fc>:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2", model_kwargs = {'device': 'cpu'})
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenti

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

54


In [ ]:
import os
import signal
import sys
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import openai


def signal_handler(sig,frame):
  print('\nThanks for using Gemini')
  sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

def generate_rag_prompt(query,context):
  escape = context.replace("'","").replace('"',"").replace("\n"," ")
  prompt = ("""
  You are a helpful and informative bot that answers questions using text from the reference context included below. \
  Be sure to repsond in a complete sentence, being comprehensive, including all relevant backgroun information. \
  However, you're talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and conversational tone. \
  If the context is irrelevant to the answer, you may ignore it.
      QUESTION: '{query}'
      CONTEXT: '{context}'

      ANSWER:
  """).format(query=query, context=context)
  return prompt

  #last line prevents hallucinations

def get_relevant_context_from_db(query):
  context = ""
  embedding_function = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2", model_kwargs = {'device': 'cpu'})
  vector_db = Chroma(persist_directory="./chroma_db_nccn", embedding_function=embedding_function)
  search_results = vector_db.similarity_search(query, k=1)
  for results in search_results:
    context += results.page_content + "\n"
  return context

OPENAI_API_KEY = 


def generate_answer(prompt):

  client = OpenAI(api_key=OPENAI_API_KEY)

  chat_completion = client.chat.completions.create(
        model="gpt-3.5-turbo",  # or "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},  # Optional: system message to set context
            {"role": "user", "content": prompt},  # User's prompt
        ],
        max_tokens=150,
        temperature=0.7
    )

    # Correctly access the content
  return chat_completion.choices[0].message.content.strip()

wrkbk = openpyxl.load_workbook("CWE-416_vul.xlsx")
sh = wrkbk.active


for i in range(1, sh.max_row + 1):
    # Extract data from the first cell in each row (or modify as needed)
    function_code = sh.cell(row=i, column=1).value
    #print(function_code)
      # Construct the query

    query = f"""Predict whether the C/C++ function below is vulnerable. Strictly return 1 for a vulnerable function or 0 for a non-vulnerable function without further explanation.

    {function_code}

    """
#     query = f"""
#     identify the CWE-ID of this vulnerability. The potential CWE-IDs
# are listed in the CWEexplanations document. Strictly
# return one of the CWE-IDs from the list
#     {function_code}

#     """

#     query = f"""
#     The following C/C++ function is confirmed vulnerable AS CWE-416, esti-
# mate the CVSS severity score (Version 3.1) of this vulnerable
# function. The potential severity score is a float between 0
# to 10. Strictly return a severity score estimation without
# any further explanation.
#     {function_code}

#     """
    #print(query)

    # query = f"""
    # Generate repair patches for the following vulnerable function.
    # {function_code[2]}

    # """
      # Assuming these functions are defined elsewhere
    context = get_relevant_context_from_db(query)
    prompt = generate_rag_prompt(query=query, context=context)
    answer = generate_answer(prompt=prompt)

      # Print or store the answer as needed
    print(f"Row {i} - Answer: {answer}")



Row 1 - Answer: I cannot determine if the C/C++ function is vulnerable based solely on the code snippet provided.
Row 2 - Answer: For the given C/C++ function, the prediction is that it is **non-vulnerable**.
Row 3 - Answer: I can't determine whether the C/C++ function is vulnerable just by looking at the code provided.
Row 4 - Answer: Based on the code provided, the C/C++ function appears to be vulnerable. I will return a value of 1 for a vulnerable function.
Row 5 - Answer: To determine if the C/C++ function provided is vulnerable, we need to look for potential issues related to memory management, such as use-after-free vulnerabilities. In this function, there is a part where the "source" pointer is being freed if the condition "free_source" is true. This action could lead to a use-after-free vulnerability, which is a type of vulnerability that can cause a program to crash, use unexpected values, or execute unintended code. Therefore, based on the context provided, this function coul